# 02_data_splitting.ipynb（データ分割）

### CSVファイルで保存

In [34]:
import os
import numpy as np
import pandas as pd

# ✅ 保存先のディレクトリ
DATA_DIR = "./data"

# ✅ 例として取得した音声特徴量（MFCC）と追加特徴量
mfccs = np.random.rand(32, 13)  # 仮のMFCCデータ（32フレーム × 13次元）
zcr = np.random.rand(32)  # ゼロ交差率
centroid = np.random.rand(32)  # スペクトルセントロイド
rms = np.random.rand(32)  # RMSエネルギー
bandwidth = np.random.rand(32)  # スペクトルバンド幅

# ✅ 特徴量データをDataFrameに格納（各行が1フレームに対応）
feature_data = {
    f"MFCC{i+1}": mfccs[:, i] for i in range(13)
}
feature_data["Zero_Crossing_Rate"] = zcr
feature_data["Spectral_Centroid"] = centroid
feature_data["RMS_Energy"] = rms
feature_data["Spectral_Bandwidth"] = bandwidth

# ✅ Pandas DataFrame に変換
df = pd.DataFrame(feature_data)

# ✅ CSVファイルとして保存
csv_path = os.path.join(DATA_DIR, "features.csv")
df.to_csv(csv_path, index=False)
print(f"✅ 音声特徴量をCSVとして保存しました: {csv_path}")

✅ 音声特徴量をCSVとして保存しました: ./data\features.csv


### CSV の保存データ取得

In [35]:
df_loaded = pd.read_csv(csv_path)
print(df_loaded.head())  # データの最初の5行を表示

      MFCC1     MFCC2     MFCC3     MFCC4     MFCC5     MFCC6     MFCC7  \
0  0.633518  0.168160  0.129056  0.080153  0.056238  0.923912  0.560058   
1  0.005679  0.144129  0.055339  0.275091  0.353391  0.538304  0.330699   
2  0.947128  0.659851  0.349774  0.612914  0.227628  0.059295  0.155140   
3  0.119872  0.734340  0.309336  0.275991  0.517493  0.280081  0.888197   
4  0.353305  0.185258  0.060374  0.652949  0.037002  0.745589  0.515441   

      MFCC8     MFCC9    MFCC10    MFCC11    MFCC12    MFCC13  \
0  0.481837  0.324598  0.575755  0.807999  0.344541  0.984036   
1  0.043408  0.713098  0.827738  0.701008  0.535437  0.069860   
2  0.869220  0.154782  0.014640  0.492886  0.188924  0.329353   
3  0.598496  0.553367  0.678009  0.880743  0.020144  0.868353   
4  0.533350  0.733237  0.901828  0.207934  0.557173  0.198347   

   Zero_Crossing_Rate  Spectral_Centroid  RMS_Energy  Spectral_Bandwidth  
0            0.618272           0.513961    0.564364            0.681241  
1       

### 特徴量とラベルを .npyで保存

In [31]:
import os
import glob
import numpy as np
import librosa
import sys

# データ保存先
DATA_DIR = "./data/"

# ラベルのディクショナリ
label_dict = {"yes": 0, "no": 1, "up": 2, "down": 3, "left": 4, "right": 5, "on": 6, "off": 7, "stop": 8, "go": 9}

# 特徴量とラベルのリスト
features = []
labels   = []

# 各フォルダ（ラベル）ごとに処理
total_files = 0  # 全ファイル数カウント
progress_symbols = ["・", "・・", "・・・", "・・・・"]  # インクリメント用記号

for label, label_id in label_dict.items():
    folder_path = os.path.join(DATA_DIR, label)
    if not os.path.exists(folder_path):
        print(f"⚠️ フォルダが見つかりません: {folder_path} → スキップ")
        continue  # フォルダが存在しない場合はスキップ

    # フォルダ内の全てのwavファイルを取得
    wav_files = glob.glob(os.path.join(folder_path, "*.wav"))

    if len(wav_files) == 0:
        print(f"⚠️ {label} フォルダに音声ファイルがありません → スキップ")
        continue

    print(f"✅ {label} フォルダ内の {len(wav_files)} ファイルを処理中...")

    for i, wav_file in enumerate(wav_files):
        try:
            # 音声ファイルを読み込み
            y, sr = librosa.load(wav_file, sr=None)
            # MFCC特徴量を抽出
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            # 平均を取って1次元ベクトルに変換
            mfcc_mean = np.mean(mfcc, axis=1)
            # 特徴量とラベルをリストに追加
            features.append(mfcc_mean)
            labels.append(label_id)
            total_files += 1

            # 進捗表示（10ファイルごとに更新）
            if (i + 1) % 10 == 0:
                symbol = progress_symbols[(i // 10) % len(progress_symbols)]
                sys.stdout.write(f"\r  ├─ {i + 1} / {len(wav_files)} ファイル処理中 {symbol} ")
                sys.stdout.flush()

        except Exception as e:
            print(f"\n❌ エラー発生: {wav_file} ({str(e)})")

    print()  # 各フォルダの処理終了後、改行

print(f"✅ 処理完了！総ファイル数: {total_files}")

# リストをNumPy配列に変換
features = np.array(features)
labels = np.array(labels)

# 特徴量とラベルを保存
features_path = os.path.join(DATA_DIR, "features.npy")
labels_path = os.path.join(DATA_DIR, "labels.npy")

np.save(features_path, features)
np.save(labels_path, labels)

# 結果を表示
print(f"✅ 特徴量とラベルの保存が完了しました。")
print(f"特徴量の形状: {features.shape} → {features_path}")
print(f"ラベルの形状: {labels.shape} → {labels_path}")

✅ yes フォルダ内の 4044 ファイルを処理中...
  ├─ 4040 / 4044 ファイル処理中 ・・・・ 
✅ no フォルダ内の 3941 ファイルを処理中...
  ├─ 3940 / 3941 ファイル処理中 ・・ ・ 
✅ up フォルダ内の 3723 ファイルを処理中...
  ├─ 3720 / 3723 ファイル処理中 ・・・・ 
✅ down フォルダ内の 3917 ファイルを処理中...
  ├─ 3910 / 3917 ファイル処理中 ・・・  
✅ left フォルダ内の 3801 ファイルを処理中...
  ├─ 3800 / 3801 ファイル処理中 ・・・・ 
✅ right フォルダ内の 3778 ファイルを処理中...
  ├─ 3770 / 3778 ファイル処理中 ・ ・・ 
✅ on フォルダ内の 3845 ファイルを処理中...
  ├─ 3840 / 3845 ファイル処理中 ・・・・ 
✅ off フォルダ内の 3745 ファイルを処理中...
  ├─ 3740 / 3745 ファイル処理中 ・・ ・ 
✅ stop フォルダ内の 3872 ファイルを処理中...
  ├─ 3870 / 3872 ファイル処理中 ・・・  
✅ go フォルダ内の 3880 ファイルを処理中...
  ├─ 3880 / 3880 ファイル処理中 ・・・・ 
✅ 処理完了！総ファイル数: 38546
✅ 特徴量とラベルの保存が完了しました。
特徴量の形状: (38546, 13) → ./data/features.npy
ラベルの形状: (38546,) → ./data/labels.npy


In [36]:
# ✅ 修正：labels.npy を正しくロードする
labels_array = np.load(labels_path)

# ロードしたデータの形状を確認
print(f"ラベルデータの形状: {labels_array.shape}")
print(f"ラベルデータのユニーク数: {np.unique(labels_array)}")

ラベルデータの形状: (38546,)
ラベルデータのユニーク数: [0 1 2 3 4 5 6 7 8 9]


### データ分割

In [37]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

DATA_DIR = "./data/"

# ✅ 保存した特徴量（MFCC）とラベルデータをロード
npy_path = os.path.join(DATA_DIR, "features.npy")
labels_path = os.path.join(DATA_DIR, "labels.npy")

X = np.load(npy_path)  # 特徴量データ
y = np.load(labels_path)  # ラベルデータ

# ✅ 特徴量データとラベルデータのサイズを合わせる
if len(y) > X.shape[0]:
    y = y[:X.shape[0]]  # ラベルを特徴量データのサイズに切り詰める
elif len(y) < X.shape[0]:
    raise ValueError(f"ラベルデータの数 ({len(y)}) が特徴量 ({X.shape[0]}) よりも少ないです。")

print(f"修正後の特徴量データ: {X.shape}")
print(f"修正後のラベルデータ: {y.shape}")

# ✅ データセットを分割（80% 訓練, 10% 検証, 10% テスト）
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# ✅ 分割結果を確認
print(f"訓練データ: {X_train.shape}, 検証データ: {X_val.shape}, テストデータ: {X_test.shape}")
print(f"訓練ラベル: {y_train.shape}, 検証ラベル: {y_val.shape}, テストラベル: {y_test.shape}")

# ✅ 各データのラベル分布を確認
print(f"訓練ラベルのユニーク値: {np.unique(y_train)}")
print(f"検証ラベルのユニーク値: {np.unique(y_val)}")
print(f"テストラベルのユニーク値: {np.unique(y_test)}")

# ✅ 分割後のデータを保存
np.save(os.path.join(DATA_DIR, "X_train.npy"), X_train)
np.save(os.path.join(DATA_DIR, "X_val.npy"), X_val)
np.save(os.path.join(DATA_DIR, "X_test.npy"), X_test)
np.save(os.path.join(DATA_DIR, "y_train.npy"), y_train)
np.save(os.path.join(DATA_DIR, "y_val.npy"), y_val)
np.save(os.path.join(DATA_DIR, "y_test.npy"), y_test)

print("✅ データセットの分割と保存が完了しました。")

修正後の特徴量データ: (38546, 13)
修正後のラベルデータ: (38546,)
訓練データ: (30836, 13), 検証データ: (3855, 13), テストデータ: (3855, 13)
訓練ラベル: (30836,), 検証ラベル: (3855,), テストラベル: (3855,)
訓練ラベルのユニーク値: [0 1 2 3 4 5 6 7 8 9]
検証ラベルのユニーク値: [0 1 2 3 4 5 6 7 8 9]
テストラベルのユニーク値: [0 1 2 3 4 5 6 7 8 9]
✅ データセットの分割と保存が完了しました。
